In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy
from scipy.ndimage import gaussian_filter1d
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error # mse}

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import os 
import glob
import cv2
import itertools

from dataloader import *


In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# LSTM Module
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm_acc = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm_gyr = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        self.reg_module1 = nn.Sequential(
            nn.Linear(hidden_size*2, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 8),
            nn.ReLU(inplace=True),
            nn.Linear(8, 1)
        )
        

    def forward(self, acc, gyr): 
        
        # 다음 학습에 영향을 주지 않기 위해 초기 h_0과 c_0 초기화
        h0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)
        print(h0.shape)
        c0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)

        _, (h_acc, _) = self.lstm_acc(acc, (h0, c0))
        _, (h_gyr, _) = self.lstm_gyr(gyr, (h0, c0))
    
        inputs_concat = torch.cat((h_acc.view(-1, hidden_size), h_gyr.view(-1, hidden_size)), dim=1)
        out_lstm = self.reg_module1(inputs_concat)
        
        return out_lstm

In [7]:
file_path = "D:\gait_dataset/salted/*"
dataset = Gait_Dataset_Salted(file_path)
val_percent = 0.2
n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val
train, val = random_split(dataset, [n_train, n_val])

In [8]:
train_loader = torch.utils.data.DataLoader(train,
                                           batch_size=128,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(val,
                                         batch_size=128,
                                         shuffle=False)

In [9]:
input_size = 300
hidden_size = 64
num_layers = 1

model = LSTM(input_size, hidden_size, num_layers).to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_epochs = 2000

loss_function = nn.MSELoss()

In [10]:
# Set fixed random number seed
torch.manual_seed(7777)

# Early Stopping을 위한 변수
best = 1000
converge_cnt = 0
total_error = 0

# Run Training loop
for epoch in range(0, n_epochs) :
    # Set current loss value 
    tot_trn_loss = 0.0
    
    # Train Mode
    model.train()
    
    # Iterate over the DataLoader for training data 
    for i, data in enumerate(train_loader) :
        inputs_acc, inputs_gyr, targets = data
        inputs_acc, inputs_gyr, targets = inputs_acc.float(), inputs_gyr.float(), targets.float()
        inputs_acc = inputs_acc.to(device)
        inputs_gyr = inputs_gyr.to(device)
        targets = targets.reshape(-1, 1)
        targets = targets.to(device)

        # 순전파 
        outputs = model(inputs_acc, inputs_gyr)
        
        # Loss 계산
        loss = loss_function(outputs, targets)
        
        # Zero the gradients 
        optimizer.zero_grad()
        # 역전파
        loss.backward()
        # Perform optimization 
        optimizer.step() 
        
        # Print statistics
        tot_trn_loss += loss.item()
        
    # Evaluation Mode
    model.eval()
    
    tot_val_loss = 0
    val_epoch_loss = []

    with torch.no_grad() :
        for i, data in enumerate(val_loader):
            inputs_acc, inputs_gyr, targets = data
            inputs_acc, inputs_gyr, targets = inputs_acc.float(), inputs_gyr.float(), targets.float()
            inputs_acc = inputs_acc.to(device)
            inputs_gyr = inputs_gyr.to(device)
            targets = targets.reshape(-1, 1)
            targets = targets.to(device)
                        
            # 순전파 
            outputs = model(inputs_acc, inputs_gyr)
            
            # Batch 별 Loss 계산
            loss = loss_function(outputs, targets)
            tot_val_loss += loss.item()            
            

    # Epoch 별 Loss
    trn_loss = tot_trn_loss / len(train_loader)
    val_loss = tot_val_loss / len(val_loader)
    error = torch.sum(torch.abs(outputs - targets) / targets) / len(targets)
    total_error += error
    
    
    print("Epoch : {}/{}, Train Loss : {:.6f}, Valid Loss {:.6f}, Error {:.6f}".format(epoch+1, n_epochs,
                                                                                       trn_loss, val_loss,
                                                                                      error))
    
    # Early Stopping
    if val_loss < best:
        best = np.mean(val_loss)
        converge_cnt = 0
    else:
        converge_cnt += 1
    
    if converge_cnt > 20:
        print('Early stopping')
        print('Total Error Mean {:4f}'.format(total_error/(epoch+1)))
        break
    
#     print("Epoch : {}/{} Epoch Loss : {:.6f}".format(epoch+1, n_epochs, current_loss / len(trainloader.dataset)))

torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 84, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 116, 64])
Epoch : 1/2000, Train Loss : 17630.685140, Valid Loss 17414.996094, Error 0.989701
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128, 64])
torch.Size([1, 128

KeyboardInterrupt: 

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# LSTM Module
class LSTM_atn(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM_atn, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm_acc = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm_gyr = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        self.reg_module1 = nn.Sequential(
            nn.Linear(hidden_size*2, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 8),
            nn.ReLU(inplace=True),
            nn.Linear(8, 1)
        )
    
    def attention(self, lstm_output, final_state):
#         merged_state = torch.cat([s for s in final_state], 1)
        merged_state = final_state.squeeze(0).unsqueeze(2)
        weights = torch.bmm(lstm_output, merged_state)
        weights = F.softmax(weights.squeeze(2), dim=1).unsqueeze(2)
        return torch.bmm(torch.transpose(lstm_output, 1, 2), weights).squeeze(2)

    def forward(self, acc, gyr): 
        
        # 다음 학습에 영향을 주지 않기 위해 초기 h_0과 c_0 초기화
        h0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)

        o_acc, (h_acc, _) = self.lstm_acc(acc, (h0, c0))
        o_gyr, (h_gyr, _) = self.lstm_gyr(gyr, (h0, c0))
        
        h_concat = torch.cat((h_acc.view(-1, hidden_size), h_gyr.view(-1, hidden_size)), dim=1)
        o_concat = torch.cat((o_acc, o_gyr), dim=2)
        
        attn_outputs = self.attention(o_concat, h_concat)
    
        out_lstm = self.reg_module1(attn_outputs)
        
        return out_lstm

In [6]:
input_size = 300
hidden_size = 64
num_layers = 1

model_atn = LSTM_atn(input_size, hidden_size, num_layers).to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model_atn.parameters(), lr=learning_rate)
n_epochs = 2000

loss_function = nn.MSELoss()

In [7]:
# Set fixed random number seed
torch.manual_seed(7777)

# Early Stopping을 위한 변수
best = 1000
converge_cnt = 0
total_error = 0

# Run Training loop
for epoch in range(0, n_epochs) :
    # Set current loss value 
    tot_trn_loss = 0.0
    
    # Train Mode
    model_atn.train()
    
    # Iterate over the DataLoader for training data 
    for i, data in enumerate(train_loader) :
        inputs_acc, inputs_gyr, targets = data
        inputs_acc, inputs_gyr, targets = inputs_acc.float(), inputs_gyr.float(), targets.float()
        inputs_acc = inputs_acc.to(device)
        inputs_gyr = inputs_gyr.to(device)
        targets = targets.reshape(-1, 1)
        targets = targets.to(device)

        # 순전파 
        outputs = model_atn(inputs_acc, inputs_gyr)
        
        # Loss 계산
        loss = loss_function(outputs, targets)
        
        # Zero the gradients 
        optimizer.zero_grad()
        # 역전파
        loss.backward()
        # Perform optimization 
        optimizer.step() 
        
        # Print statistics
        tot_trn_loss += loss.item()
        
    # Evaluation Mode
    model_atn.eval()
    
    tot_val_loss = 0
    val_epoch_loss = []

    with torch.no_grad() :
        for i, data in enumerate(val_loader):
            inputs_acc, inputs_gyr, targets = data
            inputs_acc, inputs_gyr, targets = inputs_acc.float(), inputs_gyr.float(), targets.float()
            inputs_acc = inputs_acc.to(device)
            inputs_gyr = inputs_gyr.to(device)
            targets = targets.reshape(-1, 1)
            targets = targets.to(device)
                        
            # 순전파 
            outputs = model_atn(inputs_acc, inputs_gyr)
            
            # Batch 별 Loss 계산
            loss = loss_function(outputs, targets)
            tot_val_loss += loss.item()            
            

    # Epoch 별 Loss
    trn_loss = tot_trn_loss / len(train_loader)
    val_loss = tot_val_loss / len(val_loader)
    error = torch.sum(torch.abs(outputs - targets) / targets) / len(targets)
    total_error += error
    
    print("Epoch : {}/{}, Train Loss : {:.6f}, Valid Loss {:.6f}, Error {:.6f}".format(epoch+1, n_epochs,
                                                                                       trn_loss, val_loss,
                                                                                      error))
    
    # Early Stopping
    if val_loss < best:
        best = np.mean(val_loss)
        converge_cnt = 0
    else:
        converge_cnt += 1
    
    if converge_cnt > 20:
        print('Early stopping')
        print('Total Error Mean {:4f}'.format(total_error/(epoch+1)))
        break

#     print("Epoch : {}/{} Epoch Loss : {:.6f}".format(epoch+1, n_epochs, current_loss / len(trainloader.dataset)))
            

Epoch : 1/2000, Train Loss : 17746.155599, Valid Loss 17663.092773, Error 0.996257
Epoch : 2/2000, Train Loss : 17380.334147, Valid Loss 16920.583984, Error 0.974798
Epoch : 3/2000, Train Loss : 15908.585612, Valid Loss 14371.164388, Error 0.897196
Epoch : 4/2000, Train Loss : 11838.384481, Valid Loss 8483.621582, Error 0.684904
Epoch : 5/2000, Train Loss : 4909.334173, Valid Loss 1584.802490, Error 0.271992
Epoch : 6/2000, Train Loss : 565.266740, Valid Loss 394.996328, Error 0.128506
Epoch : 7/2000, Train Loss : 329.174536, Valid Loss 340.072881, Error 0.123538
Epoch : 8/2000, Train Loss : 296.477468, Valid Loss 340.002772, Error 0.123510
Epoch : 9/2000, Train Loss : 295.836207, Valid Loss 339.882629, Error 0.123446
Epoch : 10/2000, Train Loss : 295.486719, Valid Loss 339.912120, Error 0.123472
Epoch : 11/2000, Train Loss : 295.184500, Valid Loss 339.863561, Error 0.123433
Epoch : 12/2000, Train Loss : 296.545444, Valid Loss 340.076136, Error 0.123540
Epoch : 13/2000, Train Loss : 29

Epoch : 107/2000, Train Loss : 16.216515, Valid Loss 23.194283, Error 0.027629
Epoch : 108/2000, Train Loss : 17.692392, Valid Loss 17.909806, Error 0.024385
Epoch : 109/2000, Train Loss : 16.120957, Valid Loss 16.687341, Error 0.022427
Epoch : 110/2000, Train Loss : 15.845936, Valid Loss 17.254032, Error 0.023990
Epoch : 111/2000, Train Loss : 15.293227, Valid Loss 16.690448, Error 0.023370
Epoch : 112/2000, Train Loss : 15.404807, Valid Loss 17.810793, Error 0.024242
Epoch : 113/2000, Train Loss : 16.718792, Valid Loss 16.615735, Error 0.022822
Epoch : 114/2000, Train Loss : 15.151704, Valid Loss 16.642402, Error 0.022911
Epoch : 115/2000, Train Loss : 16.668798, Valid Loss 17.154168, Error 0.024107
Epoch : 116/2000, Train Loss : 15.669579, Valid Loss 16.409843, Error 0.022480
Epoch : 117/2000, Train Loss : 15.110237, Valid Loss 19.216903, Error 0.024980
Epoch : 118/2000, Train Loss : 15.322914, Valid Loss 16.188718, Error 0.023355
Epoch : 119/2000, Train Loss : 15.127932, Valid Loss

Epoch : 211/2000, Train Loss : 11.757466, Valid Loss 19.493862, Error 0.025797
Epoch : 212/2000, Train Loss : 11.808342, Valid Loss 15.093043, Error 0.022005
Epoch : 213/2000, Train Loss : 10.690721, Valid Loss 13.177488, Error 0.021580
Epoch : 214/2000, Train Loss : 10.857700, Valid Loss 13.250388, Error 0.021363
Epoch : 215/2000, Train Loss : 10.883720, Valid Loss 13.001090, Error 0.020392
Epoch : 216/2000, Train Loss : 11.196219, Valid Loss 13.777397, Error 0.022328
Epoch : 217/2000, Train Loss : 10.523507, Valid Loss 13.140752, Error 0.021107
Epoch : 218/2000, Train Loss : 10.892065, Valid Loss 12.742526, Error 0.020511
Epoch : 219/2000, Train Loss : 10.833103, Valid Loss 15.854297, Error 0.023459
Epoch : 220/2000, Train Loss : 12.844118, Valid Loss 14.138239, Error 0.021798
Epoch : 221/2000, Train Loss : 10.867956, Valid Loss 13.381538, Error 0.021448
Epoch : 222/2000, Train Loss : 11.086153, Valid Loss 13.406042, Error 0.020780
Epoch : 223/2000, Train Loss : 11.139555, Valid Loss

Epoch : 317/2000, Train Loss : 8.077453, Valid Loss 11.687476, Error 0.019892
Epoch : 318/2000, Train Loss : 7.667007, Valid Loss 11.555269, Error 0.019508
Epoch : 319/2000, Train Loss : 7.871622, Valid Loss 11.869888, Error 0.019518
Early stopping
Total Error Mean 0.042225


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# LSTM Module
class CNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM_atn, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.cnn_acc = nn.Conv2d(in_channels=3, hidden_size, num_layers, batch_first=True)
        self.lstm_gyr = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        self.reg_module1 = nn.Sequential(
            nn.Linear(hidden_size*2, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 8),
            nn.ReLU(inplace=True),
            nn.Linear(8, 1)
        )
    
    def attention(self, lstm_output, final_state):
#         merged_state = torch.cat([s for s in final_state], 1)
        merged_state = final_state.squeeze(0).unsqueeze(2)
        weights = torch.bmm(lstm_output, merged_state)
        weights = F.softmax(weights.squeeze(2), dim=1).unsqueeze(2)
        return torch.bmm(torch.transpose(lstm_output, 1, 2), weights).squeeze(2)

    def forward(self, acc, gyr): 
        
        # 다음 학습에 영향을 주지 않기 위해 초기 h_0과 c_0 초기화
        h0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, acc.size(0), self.hidden_size).to(device)

        o_acc, (h_acc, _) = self.lstm_acc(acc, (h0, c0))
        o_gyr, (h_gyr, _) = self.lstm_gyr(gyr, (h0, c0))
        
        h_concat = torch.cat((h_acc.view(-1, hidden_size), h_gyr.view(-1, hidden_size)), dim=1)
        o_concat = torch.cat((o_acc, o_gyr), dim=2)
        
        attn_outputs = self.attention(o_concat, h_concat)
    
        out_lstm = self.reg_module1(attn_outputs)
        
        return out_lstm